dask_gateway is required to setup the connection to the cluster. Otherwise, xarray has builin support for dask so no further magic is required.dask_gateway is required to setup the connection to the cluster. Otherwise, xarray has builin support for dask so no further magic is required.

In [ ]:
%matplotlib inline

import os
import pathlib

import dask_gateway

import numpy as np
import matplotlib.pyplot as plt
import IPython.display
import xarray as xr

Connect to the the dask_gateway VM. Authentiaction is via a token available from the notebook's environment. If dask-gatway is used from the command line, it is neccessary to supply a token here.Connect to the the dask_gateway VM. Authentiaction is via a token available from the notebook's environment.

In [ ]:
gw = dask_gateway.Gateway("https://dask-gateway.jasmin.ac.uk", auth="jupyterhub")

Have the gateway create a new dask cluster. This creates a dask scheduler job in LOTUS running as the user running the notebook. Since LOTUS jobs have to queue, we re-use our cluster rather than destroying it at the end of the script.Have the gateway create a new dask cluster. This creates a dask scheduler job in LOTUS running as the user running the notebook. Since LOTUS jobs have to queue, we re-use our cluster rather than destroying it at the end of the script.

In [ ]:
clusters = gw.list_clusters()
if not clusters:
    cluster = gw.new_cluster(shutdown_on_close=False)
else:
    cluster = gw.connect(clusters[0].name)

Scale the cluster to have two workers. Each of these workers becomes it's own job in LOTUS, and is managed my the scheduler created in the last step. These workers have 4 cores and 16GiB of RAM each, but it is possible to ask for less.

In [ ]:
cluster.scale(2)

Now that we have a stood-up cluster, we can get a client with which to inteact with it.Now that we have a stood-up cluster, we can get a client with which to inteact with it.

In [ ]:
client = cluster.get_client()

Inspecting the client object will give the dashboard URL, which is proxied from the scheduler job via the gateway VM.


In [ ]:
client

Then move onto the xarray dask tutorial: https://examples.dask.org/xarray.htmlhttps://examples.dask.org/xarray.html

Opening a dataset with the 'chunks' parameter tells xarray to split the array into dask chunks.

In [ ]:
ds = xr.tutorial.open_dataset('air_temperature', chunks={'lat': 25, 'lon': 25, 'time': -1})
ds

In [ ]:
da = ds['air']
da

Resampling operations are fast because of dask.

In [ ]:
da.resample(time='1w').mean('time').std('time').load().plot(figsize=(12, 8))